In [4]:
import pandas as pd
import numpy as np
import xgboost as xgb

# Загрузка данных
df = pd.read_csv("train_aligned.csv")
df = df.iloc[:-1]  # Удаление последней строки она бракованная


# Разделяем признаки, таргет и веса
X = df.drop(columns=["target", "w"])
X = X.iloc[:,2:] # побочка
y = df["target"]
w = df["w"]

# Определение метрики WMAE
def wmae(preds, dtrain):
    y_true = dtrain.get_label()
    weights = dtrain.get_weight()
    error = np.abs(y_true - preds)
    return "WMAE", (error * weights).sum() / weights.sum()

# Создание DMatrix
dtrain = xgb.DMatrix(X, label=y, weight=w)

# Параметры модели
params = {
    "objective": "reg:squarederror",
    "max_depth": 6,
    "eta": 0.1,
    "subsample": 0.9,
    "colsample_bytree": 0.8,
    "tree_method": "hist"  # быстрый и «дружелюбный» к ноутбуку
}

# Обучение модели
model = xgb.train(
    params=params,
    dtrain=dtrain,
    num_boost_round=500,
    evals=[(dtrain, "train")],
    custom_metric=wmae,  # Используем custom_metric вместо feval
    maximize=False,
    verbose_eval=50
)

model.save_model('xgboost_model.json')
print("Модель сохранена")

# Загрузка модели
loaded_model = xgb.Booster()
loaded_model.load_model('xgboost_model.json')


df_test = pd.read_csv("test_aligned.csv")
df_test = df_test.iloc[:,2:]
df_test

# Подготовка признаков для теста (без 'target' и 'w')
X_test = df_test.drop(columns=["target", "w"], errors="ignore")

# Создание DMatrix для теста
dtest = xgb.DMatrix(X_test)

# Получение предсказаний
predictions = loaded_model.predict(dtest)

# Сохранение результатов в .csv файл
pred_df = pd.DataFrame({
    "id": df_test['id'].astype(int),  # Приводим id к целому числу
    "target": predictions
})

# Сохраняем предсказания
pred_df.to_csv("test_predictions_2.csv", index=False)